In [152]:
import pandas as pd
import pymysql

#mysql 접속정보
user_name = 'proj_user'
password = '1234'
db_name = 'mini_project_1'

conn = pymysql.connect(host='localhost', port=3306, user=user_name, passwd=password, db='mini_project_1', charset='utf8')
cursor = conn.cursor()

#mysql 읽어오기
sql = """
select b.순위, a.*
from
	(
	select 팀명, avg(ERA), avg(H), avg(SO), avg(WHIP)
		from kbo_pitcher_data
		group by 팀명
	)a
inner join kbo_team_rank as b
	on a.팀명 = b.팀명
order by 1;
"""

cursor.execute(sql)
rows = cursor.fetchall()
df = pd.DataFrame(rows, dtype=float)
df.columns = ['순위', '팀명', '평균 ERA', '평균피안타', '평균삼진', '평균출루허용']

conn.close()

df.to_csv('pitcher_stat_rank')
print(df)

corr = []
corr.append(df['순위'].corr(df['평균 ERA']))
corr.append(df['순위'].corr(df['평균피안타']))
corr.append(df['순위'].corr(df['평균삼진']))
corr.append(df['순위'].corr(df['평균출루허용']))

corr_df = pd.DataFrame(corr).T
corr_df.columns = ['평균 ERA', '평균피안타', '평균삼진', '평균출루허용']
corr_df.to_csv('pitcher_stat_corr')
corr_df


     순위   팀명     평균 ERA    평균피안타     평균삼진    평균출루허용
0   1.0  SSG   6.496429  30.7143  26.4286  1.654643
1   2.0   LG   4.217200  33.8800  28.5200  1.437600
2   3.0   키움   5.922500  36.3333  31.7083  1.701667
3   4.0   KT   4.319545  40.2727  35.2273  1.496818
4   5.0  KIA   4.664167  37.0417  30.7917  1.587917
5   6.0   롯데  10.284643  35.8214  31.0000  2.419643
6   7.0   두산   4.899130  39.4348  30.3478  1.529130
7   8.0   NC   5.516250  36.1667  34.6667  1.619167
8   9.0   삼성   4.830000  38.3846  27.2692  1.583077
9  10.0   한화   4.832333  31.9667  22.7000  1.627000


C:\Users\DETOP\AppData\Local\Temp\ipykernel_57036\917463440.py:28: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df = pd.DataFrame(rows, dtype=float)


,평균 ERA,평균피안타,평균삼진,평균출루허용
0,-0.05482,0.227375,-0.203104,0.085478


In [161]:
corr_df_concat = corr_df
for i in range(9):
    corr_df_concat = pd.concat([corr_df_concat, corr_df], ignore_index=True)
corr_df_concat

multiple_df = df.iloc[:, 2:].mul(corr_df_concat).T
result = multiple_df.sum().rank().astype('int')
result.name = '예상순위'
result_df = pd.concat([df, result], axis=1)
result_df

,순위,팀명,평균 ERA,평균피안타,평균삼진,평균출루허용,예상순위
0,1.0,SSG,6.496429,30.7143,26.4286,1.654643,2
1,2.0,LG,4.217200,33.8800,28.5200,1.437600,5
2,3.0,키움,5.922500,36.3333,31.7083,1.701667,4
3,4.0,KT,4.319545,40.2727,35.2273,1.496818,6
4,5.0,KIA,4.664167,37.0417,30.7917,1.587917,7
5,6.0,롯데,10.284643,35.8214,31.0000,2.419643,3
6,7.0,두산,4.899130,39.4348,30.3478,1.529130,9
7,8.0,NC,5.516250,36.1667,34.6667,1.619167,1
8,9.0,삼성,4.830000,38.3846,27.2692,1.583077,10
9,10.0,한화,4.832333,31.9667,22.7000,1.627000,8
